In [9]:
from demo_helper_code.demo_helper_functions import *


# German Credit Dataset:
    Dataset for a bank to predict whether or not someone is a good or bad credit risk for a loan. 
    
### Contains sensitive feature: AGE

### Input Features:
        Bank Account status, Duration of bank account, total credit history, loan purpose, total credit amount,
        savings account size, Age, Installment plants, Housing, Current Bank credits, Job skill level type,
        Number of liable people, has telephone, is foreign worker

### Output:
        0 (Bad), 1 (Good)

In [10]:
# Get German Credit Database
dataset_metadata, X, y, xTrain, xTest, yTrain, yTest = get_german_dataset()

# Represent the database in RAIs format.
rai_MetaDatabase, rai_fairness_config = get_rai_metadatabase(dataset_metadata)
rai_dataset = get_rai_dataset(xTrain, xTest, yTrain, yTest)

In [11]:
# Create a classifier and train it on the dataset.
reg, train_preds, test_preds = get_classifier_and_preds(xTrain, xTest, yTrain)

# Represent the AI Model and the Dataset as a RAI Ai System
credit_ai = get_german_rai_ai_system(reg, rai_fairness_config, rai_MetaDatabase, rai_dataset)

metric group : metadata was created
metric group : performance_cl was created
metric group : summary_stats was created
metric group : stat_moment_group was created
metric group : frequency_stats was created
metric group : correlation_stats_binary was created
metric group : basic_robustness was created
metric group : adversarial_robustness was created
metric group : group_fairness was created
metric group : individual_fairness was created
metric group : dataset_fairness was created
metric group : prediction_fairness was created
metric group : sample_distortion_fairness was created


In [12]:
# Compute Metrics
credit_ai.reset_redis()
credit_ai.compute_metrics(test_preds, data_type="test", export_title="Original")


In [13]:
# Reweigh data to accommodate for the bias against age
xTrain, xTest, yTrain, yTest = reweigh_dataset_for_age(X, y)

# Create a RAI dataset from the new reweighed data and replace the existing dataset with the new one.
new_reweigh_dataset = get_rai_dataset(xTrain, xTest, yTrain, yTest)
credit_ai.dataset = new_reweigh_dataset

In [14]:
# Retrain classifier on the new reweighed data
reg, train_preds, test_preds = get_classifier_and_preds(xTrain, xTest, yTrain, reg=reg)


# Recompute Metrics
credit_ai.compute_metrics(test_preds, data_type="test", export_title="Reweigh")